# LOGISTIC REGRESION MODEL

Cargamos el dataset resultante del eda, el cual tiene los registros con networth válido

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_parquet('./data/dota2_matches_analysed.parquet')
df.head()

,league,league_tier,league_start_date_time,league_end_date_time,league_region,series_type,match_start_date_time,match_duration_seconds,first_blood_time_seconds,radiant_team_name,...,dire_player_5_kills,dire_player_5_deaths,dire_player_5_assists,dire_player_5_networth,radiant_win,team_win,kills_diff,radiant_networth,dire_networth,networth_diff
9,European Pro League 2023-2024 Season,PROFESSIONAL,2023-10-30 22:00:00,2024-10-30 22:00:00,EUROPE,BEST_OF_THREE,2024-10-14 20:29:53,2233,0,NAVI Junior,...,1,8,7,15178,1,Radiant,6,102945,79918,23027
11,European Pro League 2023-2024 Season,PROFESSIONAL,2023-10-30 22:00:00,2024-10-30 22:00:00,EUROPE,BEST_OF_THREE,2024-10-14 19:33:03,1656,23,HYDRA,...,3,2,21,8247,0,Dire,-25,44285,77242,-32957
14,European Pro League 2023-2024 Season,PROFESSIONAL,2023-10-30 22:00:00,2024-10-30 22:00:00,EUROPE,BEST_OF_THREE,2024-10-14 18:25:18,2063,121,Team Klee,...,15,3,14,19558,0,Dire,-30,61305,87977,-26672
16,European Pro League 2023-2024 Season,PROFESSIONAL,2023-10-30 22:00:00,2024-10-30 22:00:00,EUROPE,BEST_OF_THREE,2024-10-14 17:10:53,2548,215,Team Klee,...,20,4,11,29511,0,Dire,-13,79232,107595,-28363
18,Ultras Dota Pro League 2023-24,PROFESSIONAL,2023-10-31 22:00:00,2024-10-31 22:00:00,CIS,BEST_OF_THREE,2024-10-14 16:12:43,1739,172,Ghost Sheep,...,4,7,14,13663,1,Radiant,13,85461,54045,31416


* Seleccionamos las columnas predictoras y la target

In [3]:
df.columns

Index(['league', 'league_tier', 'league_start_date_time',
       'league_end_date_time', 'league_region', 'series_type',
       'match_start_date_time', 'match_duration_seconds',
       'first_blood_time_seconds', 'radiant_team_name',
       ...
       'dire_player_5_kills', 'dire_player_5_deaths', 'dire_player_5_assists',
       'dire_player_5_networth', 'radiant_win', 'team_win', 'kills_diff',
       'radiant_networth', 'dire_networth', 'networth_diff'],
      dtype='object', length=109)

In [5]:
df_model = df[['kills_diff', 'networth_diff', 'radiant_win']].copy()

* Vemos que las columnas son las que necesitamos y están completas

In [6]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98683 entries, 9 to 134436
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   kills_diff     98683 non-null  Int16
 1   networth_diff  98683 non-null  Int64
 2   radiant_win    98683 non-null  int64
dtypes: Int16(1), Int64(1), int64(1)
memory usage: 2.6 MB


* Seleccionamos nuestras variables predictoras y la objetivo

In [10]:
# selecting X and y
X = df_model.drop('radiant_win', axis=1)
y = df_model['radiant_win']

* Separaremos los registros solo en registros de entrenamiento y prueba (los de evaluación descartaremos ya que nos hemos quedado con muy pocos registros y además por la simplicidad del proyecto)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

* Aplicamos técnica de estandarización para las variables predictoras de entrenamiento y prueba

In [16]:
from sklearn.preprocessing import StandardScaler

sc_model = StandardScaler()

X_train_scaled = sc_model.fit_transform(X_train)
X_test_scaled = sc_model.transform(X_test)

* Entrenamos al modelo de regresión logística

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(X_train_scaled, y_train)

LogisticRegression()

* Usamos al modelo para realizar las predicciones en nuestros datos de prueba

In [22]:
y_pred = model.predict(X_test_scaled)

## Evaluando desempeño del modelo

### Accuracy Score

In [23]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.9881440948472412

* Esta métrica es muy general y solo nos da una idea, no podemos confiar tanto en ella ya que puede fallar si la distribución de nuestra variable objetivo (radiant_win) está desbalanceada

## Confussion Matrix

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[9596,  122],
       [ 112, 9907]])

Esta métrica muestra el rendimiento del modelo mostrando exactamente cuántas predicciones fueron correctas e incorrectas, para cada clase en un array, como es predicción binaria es un 2x2

## Classification Report

El classification_report proporciona un resumen conciso del rendimiento por clase. Calcula la Precision (qué proporción de predicciones para una clase fueron correctas), el Recall (qué proporción de las instancias reales de una clase se detectaron) y el F1-Score (media armónica de precision y recall), además del Support (número de muestras reales por clase). Es muy útil para evaluar el equilibrio del modelo entre diferentes tipos de errores (source: ai studio)

In [29]:
from sklearn.metrics import classification_report

# cr = classification_report(y_test, y_pred, output_dict=True)
# pd.DataFrame(cr).transpose()
classification_report(y_test, y_pred)


'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99      9718\n           1       0.99      0.99      0.99     10019\n\n    accuracy                           0.99     19737\n   macro avg       0.99      0.99      0.99     19737\nweighted avg       0.99      0.99      0.99     19737\n'

## AUC - ROC

Esta métrica mide qué tan bien el modelo puede distinguir entre las dos clases (radiant/dire win), basándose en las probabilidades predichas. Un valor de 1 es perfecto, 0.5 es como adivinar al azar. Nuestro AUC cercano a 1 (ej. ~0.998) indica una excelente capacidad de discriminación.

1. Obtener Probabilidades Predichas de nuestro modelo de regresión logística

In [33]:
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
y_pred_proba[:5]

array([8.86360158e-04, 1.24603384e-03, 9.99930197e-01, 9.96570168e-01,
       4.99516403e-08])

2. Importar e imprimir la métrica

In [34]:
from sklearn.metrics import roc_auc_score

# Calculate the AUC score
auc_score = roc_auc_score(y_test, y_pred_proba)
auc_score

np.float64(0.9978738123434994)

Todas las métricas coincidieron en que el modelo es muy preciso y fiable para predecir si radiant o dire gana. Esto confirma la fuerte relación que vimos en el EDA entre las diferencias de kills y networth (`kills_diff`, `networth_diff`) y el equipo ganador.